## Análise Resultados Corrida

1.   Importação dos resultados
2.   Limpeza dos dados
3.   Média Swim, Bike, Run por categoria
4.   Melhores Splits Swim, Bike, Run por categoria
5.   Quantidade de Atleats por aCtegoria
6.   Percentual de Sub10 geral
7.   Percentual de Sub10 por categoria amadora

In [0]:
import numpy as np
import pandas as pd

missing_values = ['--:--:--']
resultados = pd.read_csv('http://doricardo.com/results/2018-uruguay70.3-full.csv', na_values = missing_values, encoding = 'cp1252')
target = '05'
continent = pd.read_csv('http://doricardo.com/dataframe/country-and-continent.csv')
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']

In [0]:
def limpar_titulo(dataframe):
    dataframe.drop(['Pos'])   
    
def recuperar_media(df, categoria, sexo, atividade):
    tempos = pd.to_datetime(df[(df.Categ == categoria) & (df.Sexo == sexo) & (df[atividade].notnull())][atividade], format= '%H:%M:%S').astype(int).mean()
    media = pd.to_datetime(tempos, format= '%Y-%m-%d %H:%M:%S')
    return pd.to_datetime(media, format= '%Y-%m-%d %H:%M:%S')

In [0]:
# Limpar nome Colunas duplicadas
resultados = resultados[resultados.Pos != 'Pos']

# Incluir nome Colunas
resultados.columns = ['Bib', 'Pos', 'Nome', 'Pais', 'Categ', 'Pos Categ', 'Swim', 'Bike', 'Run', 'Total', 'Status', 'Sexo']

In [0]:
resultados['Finished'] = resultados['Status']

In [0]:
resultados.loc[resultados.Status != 'Finish', 'Finished'] = 'No'
resultados.loc[resultados.Status == 'Finish', 'Finished'] = 'Yes'

In [0]:
finished = resultados[resultados.Status == 'Finish']
unfinished = resultados[resultados.Status != 'Finish']

In [7]:
finished.Swim.fillna(pd.to_datetime(pd.to_datetime(finished['Swim'],format= '%H:%M:%S').astype(int).median(), format= '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S'), inplace=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [0]:
finished.Bike.fillna(pd.to_datetime(pd.to_datetime(finished['Bike'],format= '%H:%M:%S').astype(int).median(), format= '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S'), inplace=True)

In [0]:
finished.Run.fillna(pd.to_datetime(pd.to_datetime(finished['Run'],format= '%H:%M:%S').astype(int).median(), format= '%Y-%m-%d %H:%M:%S').strftime('%H:%M:%S'), inplace=True)

In [0]:
# Percentual de atletas que Concluiram a prova
resultados.groupby(['Finished']).Pos.count().plot.pie(title='', colors=colors, shadow=True, startangle=90, autopct='%1.0f%%').axis('equal');

In [0]:
# Distribuição dos motivos de não conlusão da prova
unfinished.groupby(['Status']).Pos.count().plot.pie(title='', colors=colors, shadow=True, startangle=90, autopct='%1.0f%%').axis('equal');

In [0]:
hist = finished.groupby(['Categ','Sexo']).Bib.count().unstack().plot.barh(rot = 0, title='Categorias por Genero Ironamn 2018', color=colors)
hist.set_ylabel('')
hist.set_xlabel('Quantidade');

In [0]:
finished['hora'] = np.where(finished.Total.str.slice(0, 2) < target, 'Abaixo' + target, 'Acima' + target)

In [0]:
finished.groupby(['hora']).Pos.count().plot.pie(title='', colors=colors, shadow=True, startangle=90, explode=[0.1, 0], autopct='%1.0f%%').axis('equal');

##Converter Colunas

In [0]:
resultados['Swim'] = pd.to_datetime(resultados['Swim'],format= '%H:%M:%S')
resultados['Bike'] = pd.to_datetime(resultados['Bike'],format= '%H:%M:%S')
resultados['Run'] = pd.to_datetime(resultados['Run'],format= '%H:%M:%S')
resultados['Total'] = pd.to_datetime(resultados['Total'],format= '%H:%M:%S')

In [0]:
resultados['nSwim'] = pd.to_datetime(resultados['Swim'],format= '%H:%M:%S').astype(int)
resultados['nBike'] = pd.to_datetime(resultados['Bike'],format= '%H:%M:%S').astype(int)
resultados['nRun'] = pd.to_datetime(resultados['Run'],format= '%H:%M:%S').astype(int)
resultados['nTotal'] = pd.to_datetime(resultados['Total'],format= '%H:%M:%S').astype(int)

##Estatisticas Colunas

In [0]:
media = resultados.groupby(['Categ']).mean()

In [0]:
media['Total'] = pd.to_datetime(media['nTotal'], format= '%Y-%m-%d %H:%M:%S')
media['Swim'] = pd.to_datetime(media['nSwim'], format= '%Y-%m-%d %H:%M:%S')
media['Bike'] = pd.to_datetime(media['nBike'], format= '%Y-%m-%d %H:%M:%S')
media['Run'] = pd.to_datetime(media['nRun'], format= '%Y-%m-%d %H:%M:%S')

In [0]:
media

In [0]:
mais_rapidos = finished.groupby(['Categ','Sexo']).min()

In [0]:
mais_rapidos

In [0]:
#mais_rapidos['Total'] = pd.to_datetime(mais_rapidos['nTotal'], format= '%Y-%m-%d %H:%M:%S')
#mais_rapidos['Swim'] = pd.to_datetime(mais_rapidos['nSwim'], format= '%Y-%m-%d %H:%M:%S')
#mais_rapidos['Bike'] = pd.to_datetime(mais_rapidos['nBike'], format= '%Y-%m-%d %H:%M:%S')
#mais_rapidos['Run'] = pd.to_datetime(mais_rapidos['nRun'], format= '%Y-%m-%d %H:%M:%S')

In [0]:
mais_rapidos[['Swim','Bike','Run','Total','Nome']]

In [0]:
#categorias_excluidas = ['C','PRO']
#categorias = resultados[~resultados.isin(categorias_excluidas)] 
categorias = resultados
hist = categorias['Categ'].value_counts().sort_index(ascending=True).plot.bar(rot=0, title="Quantidade Atletas por Categoria", figsize=(10,6) )

rects = hist.patches

# Make labels
labels = [ i for i in categorias['Categ'].value_counts().sort_index(ascending=True) ]

for rect, label in zip(rects, labels):
    height = rect.get_height()
    hist.text(rect.get_x() + rect.get_width() / 2, height, label, ha='center', va='bottom')

hist.patch.set_facecolor('#FFFFFF')

In [0]:
resultados.loc[resultados['Total'] < pd.to_datetime(target + ':00:00', format= '%H:%M:%S'), 'Abaixo' + target] = 'Abaixo ' + target + ' horas' 
resultados.loc[resultados['Total'] >= pd.to_datetime(target + ':00:00', format= '%H:%M:%S'), 'Abaixo' + target] = 'Acima ' + target + ' horas' 

In [0]:
resultados['Abaixo' + target].value_counts().plot.pie(title='', explode=[0.1, 0], autopct='%1.0f%%');